# DATA PREPARATION LAST FM

FICHIER: *'userid-timestamp-artid-artname-traid-traname.tsv'*

## Imports 

In [ ]:
import pandas as pd
import numpy as np

## Data import

In [19]:
df = pd.read_csv('filtered data/filtered_fm_dataset.csv', error_bad_lines=False)
df = df.iloc[:,1:]

C:\Users\Alumet\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


### Calcul du nomnre d'écoute pour un track_id par utilisateur

In [20]:
df_track = df.groupby(['user_id','track_id']).size().reset_index(name='listening_count')

In [22]:
g = df_track.groupby('user_id')["listening_count"]
min_, max_ = g.transform('min'), g.transform('max')

### Normalisation du nombre d'écoutes 

In [23]:
df_track["listening_count_normalised"] = (df_track["listening_count"] - min_) / (max_ - min_)*100
df_track["listening_count_normalised"] = df_track["listening_count_normalised"].fillna(0.0).astype(int)+1

,user_id,track_id,listening_count,listening_count_normalised
0,1,0,7,9
1,1,1,8,11
2,1,2,8,11
3,1,3,7,9
4,1,4,6,8
...,...,...,...,...
2152804,1000,80283,3,4
2152805,1000,80414,1,1
2152806,1000,80427,1,1
2152807,1000,80500,1,1


In [24]:
df_final_jg=df_track[["user_id","track_id","listening_count_normalised"]]
df_final_jg=df_final_jg.rename(columns={"listening_count_normalised":"rating"})

,user_id,track_id,rating
0,1,0,9
1,1,1,11
2,1,2,11
3,1,3,9
4,1,4,8
...,...,...,...
2152804,1000,80283,4
2152805,1000,80414,1
2152806,1000,80427,1
2152807,1000,80500,1


In [25]:
df_rating = pd.merge(
            df,
            df_final_jg,
            how="left",
            left_on=['user_id', 'track_id'],
            right_on=['user_id', 'track_id']
        )

In [26]:
df_rating.head()

,user_id,time_stamp,artist_id,artist_name,track_id,track_name,rating
0,1,2009-05-03T15:10:18Z,4,Minus 8,0,Elysian Fields,9
1,1,2009-05-03T15:04:31Z,5,Beanfield,1,Planetary Deadlock,11
2,1,2009-05-03T14:50:51Z,7,Alif Tree,2,Deadly Species,11
3,1,2009-05-03T14:46:29Z,4,Minus 8,3,Cold Fusion,9
4,1,2009-05-02T15:00:59Z,14,4Hero,4,Look Inside,8


In [27]:
df.head()

,user_id,time_stamp,artist_id,artist_name,track_id,track_name
0,1,2009-05-03T15:10:18Z,4,Minus 8,0,Elysian Fields
1,1,2009-05-03T15:04:31Z,5,Beanfield,1,Planetary Deadlock
2,1,2009-05-03T14:50:51Z,7,Alif Tree,2,Deadly Species
3,1,2009-05-03T14:46:29Z,4,Minus 8,3,Cold Fusion
4,1,2009-05-02T15:00:59Z,14,4Hero,4,Look Inside


In [28]:
df_rating.to_csv('filtered data/filtered_rating_fm_dataset.csv',index=False,float_format='%.2f')